# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import re
import pickle
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download(["wordnet", "punkt", "stopwords"])

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, classification_report

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\btian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\btian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\btian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine("sqlite:///disaster_messages_database.db")
df = pd.read_sql_table("disaster_messages_table", engine)
X = df.message
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Function:
    Split sentences into words then return the root form of the words (or lemmatized words)
    
    Args:
    text(str): the input message
    
    Return:
    lemmed(list): a list of lemmatized words
    """
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize text
    words = word_tokenize(text)
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    # Lemmatize words
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# Pipeline 1: AdaBoost Classifier
pipeline_ab = Pipeline([
    ("vect", CountVectorizer(tokenizer = tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(AdaBoostClassifier()))
])

# Pipeline 2: Naive Bayes Classifier
pipeline_nb = Pipeline([
    ("vect", CountVectorizer(tokenizer = tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(MultinomialNB()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [7]:
# Fit AdaBoost classifier
pipeline_ab.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000024C4AEA2E50>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [8]:
# Fit naive bayes classifier
pipeline_nb.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000024C4AEA2E50>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=MultinomialNB()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# Test performance for AdaBoost classifier
y_pred_ab = pipeline_ab.predict(X_test)

In [10]:
len(y_test.columns)

36

In [15]:
# Print results
for i in range(len(y_test.columns)):
    print(y_test.columns[i], ":")
    print(classification_report(y_test[y_test.columns[i]], y_pred_ab[:, i]))
    
accuracy_ab = (y_pred_ab == y_test.values).mean()
print("The overall accuracy: {:.3f}".format(accuracy_ab))

related :
              precision    recall  f1-score   support

           0       0.67      0.15      0.25      1588
           1       0.78      0.97      0.86      4914
           2       0.40      0.19      0.26        52

    accuracy                           0.77      6554
   macro avg       0.62      0.44      0.46      6554
weighted avg       0.75      0.77      0.71      6554

request :
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5479
           1       0.74      0.48      0.59      1075

    accuracy                           0.89      6554
   macro avg       0.82      0.73      0.76      6554
weighted avg       0.88      0.89      0.88      6554

offer :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50   

In [12]:
# Test performance for naive bayes classifier
y_pred_nb = pipeline_nb.predict(X_test)

In [16]:
# Print results
for i in range(len(y_test.columns)):
    print(y_test.columns[i], ":")
    print(classification_report(y_test[y_test.columns[i]], y_pred_nb[:, i]))
    
accuracy_nb = (y_pred_nb == y_test.values).mean()
print("The overall accuracy: {:.3f}".format(accuracy_nb))

related :
              precision    recall  f1-score   support

           0       0.79      0.09      0.16      1588
           1       0.77      0.99      0.86      4914
           2       0.00      0.00      0.00        52

    accuracy                           0.77      6554
   macro avg       0.52      0.36      0.34      6554
weighted avg       0.76      0.77      0.69      6554

request :
              precision    recall  f1-score   support

           0       0.87      0.99      0.92      5479
           1       0.84      0.21      0.34      1075

    accuracy                           0.86      6554
   macro avg       0.85      0.60      0.63      6554
weighted avg       0.86      0.86      0.83      6554

offer :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50   

C:\Users\btian\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\btian\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\btian\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C

### 6. Improve your model
Use grid search to find better parameters.

In [14]:
pipeline_ab.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000024C4AEA2E50>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000024C4AEA2E50>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [17]:
parameters_ab = {
    "tfidf__use_idf": (True, False),
    "clf__estimator__n_estimators": [50, 70, 80]
}

cv_ab = GridSearchCV(pipeline_ab, param_grid = parameters_ab)

In [18]:
cv_ab

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000024C4AEA2E50>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 70, 80],
                         'tfidf__use_idf': (True, False)})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
cv_ab.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000024C4AEA2E50>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 70, 80],
                         'tfidf__use_idf': (True, False)})

In [22]:
print(cv_ab.best_params_)

{'clf__estimator__n_estimators': 80, 'tfidf__use_idf': False}


In [23]:
optimized_model = cv_ab.best_estimator_
print(optimized_model)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000024C4AEA2E50>)),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(n_estimators=80)))])


In [24]:
y_pred_op = optimized_model.predict(X_test)

In [25]:
for i in range(len(y_test.columns)):
    print(y_test.columns[i], ":")
    print(classification_report(y_test[y_test.columns[i]], y_pred_op[:, i]))
    
accuracy_op = (y_pred_op == y_test.values).mean()
print("The overall accuracy: {:.3f}".format(accuracy_op))

related :
              precision    recall  f1-score   support

           0       0.63      0.12      0.20      1588
           1       0.77      0.98      0.86      4914
           2       0.44      0.21      0.29        52

    accuracy                           0.76      6554
   macro avg       0.61      0.44      0.45      6554
weighted avg       0.73      0.76      0.70      6554

request :
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      5479
           1       0.74      0.54      0.62      1075

    accuracy                           0.89      6554
   macro avg       0.83      0.75      0.78      6554
weighted avg       0.89      0.89      0.89      6554

offer :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50   

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6480
           1       0.49      0.24      0.32        74

    accuracy                           0.99      6554
   macro avg       0.74      0.62      0.66      6554
weighted avg       0.99      0.99      0.99      6554

earthquake :
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5918
           1       0.87      0.81      0.84       636

    accuracy                           0.97      6554
   macro avg       0.92      0.90      0.91      6554
weighted avg       0.97      0.97      0.97      6554

cold :
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6411
           1       0.68      0.30      0.42       143

    accuracy                           0.98      6554
   macro avg       0.83      0.65      0.70      6554
weighted avg       0.98      0.98      0.98      6554


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [26]:
# Ouput a pickle file for the optimized classifier
with open ("classifier.pkl", "wb") as f:
    pickle.dump(optimized_model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.